In [100]:
import pandas as pd
from pathlib import Path
from rdkit.Chem import AllChem
from tqdm import tqdm

from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*')

In [101]:
fold = 0

test_src = Path(f"data/radicals/radicals_fold{fold}_test_src.txt")
test_src_pred = Path(f"data/radicals/uspto_radicals_fold{fold}_test_src_pred.pickle")

df_test_src_pred = pd.read_pickle(test_src_pred)
df_test_src = pd.read_csv(test_src, header=None, names=["src"])

In [102]:
df_test_src_pred.head()

,original_smiles,prediction_0,log_likelihood_0,prediction_1,log_likelihood_1,prediction_2,log_likelihood_2,prediction_3,log_likelihood_3,prediction_4,...,prediction_5,log_likelihood_5,prediction_6,log_likelihood_6,prediction_7,log_likelihood_7,prediction_8,log_likelihood_8,prediction_9,log_likelihood_9
0,C[C](C)C(Sc1ccccc1)=C(C)C,CC(C)=C=C(C)C.c1ccc(SSc2ccccc2)cc1,-0.647619,c1(SSc2ccccc2)ccccc1.CC(=C=C(C)C)C,-4.087032,c1c(SSc2ccccc2)cccc1.CC(=C=C(C)C)C,-4.131845,c1cccc(SSc2ccccc2)c1.CC(=C=C(C)C)C,-4.168436,c1cc(SSc2ccccc2)ccc1.CC(=C=C(C)C)C,...,c1(SSc2ccccc2)ccccc1.CC(C)=C=C(C)C,-4.194435,c1ccc(SSc2ccccc2)cc1.CC(=C=C(C)C)C,-4.218727,c1c(SSc2ccccc2)cccc1.CC(C)=C=C(C)C,-4.237559,c1cccc(SSc2ccccc2)c1.CC(C)=C=C(C)C,-4.273448,c1cc(SSc2ccccc2)ccc1.CC(C)=C=C(C)C,-4.295864
1,C[C](C)CCC#N,C=C(C)C.C=C=[N],-0.559250,C=C=[N].C=C(C)C,-2.400765,CC(C)=C.C=C=[N],-3.807516,CC(=C)C.C=C=[N],-3.948586,C(=C)=[N].C=C(C)C,...,C(=C)(C)C.C=C=[N],-4.700393,C(=C)=[N].CC(=C)C,-4.868047,CC(C)=C.C(=[N])=C,-4.970725,CC(=C)C.C(=[N])=C,-4.993320,C(=C)(C)C.C(=C)=[N],-5.551546
2,CC[C](CC1CCCCC1)C(=O)OC,C=C(CC)C(=O)OC.[CH]1CCCCC1,-1.142110,C1CCCCC1.C=C(CC)C(=O)OC,-3.500899,C1CC[CH]CC1.C=C(CC)C(=O)OC,-5.182951,C1CC[CH]CC1.C=C(CC)C(OC)=O,-5.251920,C1CCCC[CH]1.C=C(CC)C(OC)=O,...,C1CCCC[CH]1.C=C(CC)C(=O)OC,-5.365485,C1CC[CH]CC1.C=C(C(=O)OC)CC,-5.995557,C1CCCC[CH]1.C=C(C(=O)OC)CC,-6.031199,C1CC[CH]CC1.C(C)C(C(OC)=O)=C,-6.479540,C1CC[CH]CC1.C(C)C(=C)C(OC)=O,-6.525710
3,CCCC[C](CC)CCCC,[CH2]CCCC(CC)CCCC,-0.528338,CCCCC(CC)CCC[CH2],-1.989608,CCC(CCCC)CCC[CH2],-3.995576,C(CCC(CC)CCC[CH2])C,-4.244432,C(CCC)C(CC)CCC[CH2],...,C(CC(CC)CCC[CH2])CC,-4.332236,C(CC)CC(CC)CCC[CH2],-4.524948,C(C(CC)CCC[CH2])CCC,-4.552934,C(C)CCC(CC)CCC[CH2],-4.570644,C(CCC(CCC[CH2])CC)C,-4.780808
4,CCCCC[C](CCCC)CCCC,[CH2]CCCC(CCCC)CCCCC,-0.574412,CCCCC(CCCC)CCC[CH2],-2.921343,CCCCC(CCC[CH2])CCCCC,-3.220763,[CH2]CCCC(CCCCC)CCCCC,-3.267129,CCCCCC(CCC[CH2])CCCCC,...,CCCCCC(CCCC)CCC[CH2],-3.963874,C(CCC(CCCC)CCC[CH2])C,-4.216188,C(CC(CCCC)CCC[CH2])CCC,-4.409172,C(C(CCCC)CCC[CH2])CCCC,-4.463913,C(CC)CCC(CCC[CH2])CCCCC,-4.538358


In [103]:
df_test_src.head()

,src
0,CC(C)=C=C(C)C.c1ccc(SSc2ccccc2)cc1
1,C=C(C)C.C=C=[N]
2,C=C(CC)C(=O)OC.[CH]1CCCCC1
3,[CH2]CCCC(CC)CCCC
4,[CH2]CCCC(CCCC)CCCCC


In [104]:
# concat dfs
assert len(df_test_src_pred) == len(df_test_src)
df = df_test_src_pred.copy()
df['src'] = df_test_src['src']
df.columns

Index(['original_smiles', 'prediction_0', 'log_likelihood_0', 'prediction_1',
       'log_likelihood_1', 'prediction_2', 'log_likelihood_2', 'prediction_3',
       'log_likelihood_3', 'prediction_4', 'log_likelihood_4', 'prediction_5',
       'log_likelihood_5', 'prediction_6', 'log_likelihood_6', 'prediction_7',
       'log_likelihood_7', 'prediction_8', 'log_likelihood_8', 'prediction_9',
       'log_likelihood_9', 'src'],
      dtype='object')

In [105]:
df.tail()

,original_smiles,prediction_0,log_likelihood_0,prediction_1,log_likelihood_1,prediction_2,log_likelihood_2,prediction_3,log_likelihood_3,prediction_4,...,log_likelihood_5,prediction_6,log_likelihood_6,prediction_7,log_likelihood_7,prediction_8,log_likelihood_8,prediction_9,log_likelihood_9,src
1755,CCOc1ccc(-c2ccc(C(C)CC(=O)OC)cc2)cc1,CC=CC(=O)OC.CCOc1ccc(-c2ccc(B(O)O)cc2)cc1,-0.692673,c1(B(O)O)ccc(-c2ccc(OCC)cc2)cc1.COC(=O)C=CC,-5.644981,c1(B(O)O)ccc(-c2ccc(OCC)cc2)cc1.CC=CC(OC)=O,-5.652134,c1c(B(O)O)ccc(-c2ccc(OCC)cc2)c1.COC(=O)C=CC,-5.657983,c1c(B(O)O)ccc(-c2ccc(OCC)cc2)c1.CC=CC(OC)=O,...,-5.697880,c1c(B(O)O)ccc(-c2ccc(OCC)cc2)c1.COC(C=CC)=O,-5.711237,c1(B(O)O)ccc(-c2ccc(OCC)cc2)cc1.CC=CC(=O)OC,-5.716335,c1c(B(O)O)ccc(-c2ccc(OCC)cc2)c1.CC=CC(=O)OC,-5.741049,c1(B(O)O)ccc(-c2ccc(OCC)cc2)cc1.O(C)C(=O)C=CC,-6.290987,CC=CC(=O)OC.CCOc1ccc(-c2ccc(B(O)O)cc2)cc1
1756,CCOc1cccc(-c2ccc(CCC(C)=O)cc2)c1,C=CC(C)=O.CCOc1cccc(-c2ccc(B(O)O)cc2)c1,-0.733953,c1(B(O)O)ccc(-c2cccc(OCC)c2)cc1.O=C(C)C=C,-6.232986,c1c(B(O)O)ccc(-c2cccc(OCC)c2)c1.O=C(C)C=C,-6.243717,c1c(B(O)O)ccc(-c2cccc(OCC)c2)c1.CC(C=C)=O,-6.274298,c1(B(O)O)ccc(-c2cccc(OCC)c2)cc1.CC(C=C)=O,...,-6.284818,c1(B(O)O)ccc(-c2cccc(OCC)c2)cc1.O=C(C=C)C,-6.358708,c1c(B(O)O)ccc(-c2cccc(OCC)c2)c1.O=C(C=C)C,-6.385908,c1(B(O)O)ccc(-c2cccc(OCC)c2)cc1.C=CC(=O)C,-6.424369,c1(B(O)O)ccc(-c2cccc(OCC)c2)cc1.C(C)(C=C)=O,-6.998495,C=CC(C)=O.CCOc1cccc(-c2ccc(B(O)O)cc2)c1
1757,CCO[C]1CC1,CCOC1(Br)CC1,-1.316846,CCOC1(O[Si](C)(C)C)CC1,-1.483987,O(C1(O[Si](C)(C)C)CC1)CC,-3.070269,O(CC)C1(Br)CC1,-3.186790,O(CC)C1(O[Si](C)(C)C)CC1,...,-3.530128,BrC1(OCC)CC1,-3.597192,C1CC1(Br)OCC,-3.685864,C1CC1(OCC)Br,-3.973289,C1(OCC)(Br)CC1,-4.086560,CCOC1(Br)CC1
1758,C[C](C)c1ccccc1C1(C)[C](c2ccccc2)C1c1ccccc1,CC(C)c1ccccc1C1(C)C(c2ccccc2)=C1c1ccccc1,-0.632108,C1(C)(c2ccccc2C(C)C)C(c2ccccc2)=C1c1ccccc1,-4.137080,C1(c2ccccc2)=C(c2ccccc2)C1(C)c1c(C(C)C)cccc1,-4.368083,C1(c2ccccc2)=C(c2ccccc2)C1(C)c1ccccc1C(C)C,-4.428292,c1cccc(C2=C(c3ccccc3)C2(C)c2ccccc2C(C)C)c1,...,-4.529862,c1ccccc1C1=C(c2ccccc2)C1(C)c1c(C(C)C)cccc1,-4.618185,c1cccc(C2=C(c3ccccc3)C2(c2ccccc2C(C)C)C)c1,-4.648948,c1ccccc1C1=C(c2ccccc2)C1(C)c1ccccc1C(C)C,-4.660923,c1ccc(C2=C(c3ccccc3)C2(C)c2ccccc2C(C)C)cc1,-4.669493,CC(C)c1ccccc1C1(C)C(c2ccccc2)=C1c1ccccc1
1759,[CH2]c1ccccc1C1(C)[C](c2ccccc2)C1c1ccccc1,Cc1ccccc1C1(C)C(c2ccccc2)=C1c1ccccc1,-0.664679,c1(C2=C(c3ccccc3)C2(c2ccccc2C)C)ccccc1,-3.915291,c1cccc(C2=C(c3ccccc3)C2(c2ccccc2C)C)c1,-3.995283,c1ccc(C2=C(c3ccccc3)C2(c2ccccc2C)C)cc1,-4.123190,c1cccc(C2=C(c3ccccc3)C2(c2c(C)cccc2)C)c1,...,-4.295036,c1ccccc1C1=C(c2ccccc2)C1(c1ccccc1C)C,-4.369150,c1cccc(C2=C(c3ccccc3)C2(C)c2c(C)cccc2)c1,-4.374035,C1(c2ccccc2)=C(c2ccccc2)C1(c1ccccc1C)C,-4.749811,c1ccccc1C1=C(c2ccccc2)C1(C)c1ccccc1C,-4.759606,Cc1ccccc1C1(C)C(c2ccccc2)=C1c1ccccc1


In [106]:
def is_valid(smi: str, ignore=True):
    try:
        mol = AllChem.MolFromSmiles(smi)
        if mol is None:
            return False
        else:
            return True
    except Exception as e:
        if not ignore:
            print(f'Not valid: {e}')
        return False

def cononize(smi: str, ignore=True):
    """标准化

    Args:
        smi (str): 分子 SMILES

    Returns:
        _type_: None
    """
    try:
        return AllChem.MolToSmiles(AllChem.MolFromSmiles(smi), isomericSmiles=False, canonical=True)
    except Exception as e:
        if not ignore:
            print(f'Failed Canonizing: {e}')
        return None

In [107]:
def print_topn_acc(df, top_n):
    df = df.copy()
    n_correct = 0
    n_valid = 0
    top_n = top_n
    for i, row in tqdm(df.iterrows(), total=len(df)):
        src = cononize(row['src'])
        for j in range(top_n):
            if is_valid(row[f'prediction_{j}']):
                n_valid += 1
        for k in range(top_n):
            if src == cononize(row[f'prediction_{k}']):
                n_correct += 1
                break
    print('-------------->')
    print(f'top {top_n} acc:')
    print(f'valid rate: {n_valid / len(df) / top_n:.2f}')
    print(f'acc: {n_correct / len(df):.2f}')
    print('<--------------')

In [108]:
for i in [1, 3, 5, 10]:
    print(i)
    print_topn_acc(df, top_n=i)

1


100%|██████████| 1760/1760 [00:00<00:00, 1848.99it/s]


-------------->
top 1 acc:
valid rate: 0.98
acc: 0.91
<--------------
3


100%|██████████| 1760/1760 [00:01<00:00, 1280.57it/s]


-------------->
top 3 acc:
valid rate: 0.98
acc: 0.97
<--------------
5


100%|██████████| 1760/1760 [00:01<00:00, 1001.25it/s]


-------------->
top 5 acc:
valid rate: 0.98
acc: 0.98
<--------------
10


100%|██████████| 1760/1760 [00:02<00:00, 654.64it/s]

-------------->
top 10 acc:
valid rate: 0.98
acc: 0.98
<--------------
